In [ ]:
%%capture
!pip install datasets
!pip install bert_score
!pip install rouge_score

In [ ]:
%%capture
!pip install blanc

In [ ]:
from datasets import load_metric
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
bertscore = load_metric("bertscore",trust_remote_code=True)
meteor = load_metric("meteor",trust_remote_code=True)
bleu = load_metric("bleu",trust_remote_code=True)
rouge = load_metric('rouge',trust_remote_code=True)

<ipython-input-4-bd738deae4d8>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bertscore = load_metric("bertscore",trust_remote_code=True)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def calculate_bleu_score(candidate, references):
    candidate_tokens = nltk.word_tokenize(candidate)
    reference_tokens = [nltk.word_tokenize(ref) for ref in references]
    # Using a smoothing function for cases where n-gram matches result in 0
    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothie)
    return bleu_score

In [ ]:
def metrics(actual, pred):
  predictions = [pred]
  references = [actual]
  metrics = {}
  #Rouge
  rouge_score = rouge.compute(predictions=predictions, references=references, use_aggregator=False)
  metrics["rouge"] = [{
      "rouge1": rouge_score["rouge1"][0].fmeasure,
      "rouge2": rouge_score["rouge2"][0].fmeasure,
      "rougeL": rouge_score["rougeL"][0].fmeasure
  }]
  #BERT
  bert_score = bertscore.compute(predictions=predictions, references=references, model_type="bert-base-uncased")
  metrics["bert"] = bert_score["f1"][0]
  #Meteor
  meteor_score = meteor.compute(predictions=predictions, references=references)
  metrics["meteor"] = meteor_score["meteor"]
  #BLEU
  bleu_score = calculate_bleu_score(predictions[0], references)
  metrics["bleu"] = bleu_score
  return metrics

In [ ]:
import pandas as pd
df = pd.read_csv("prediction_explanation.csv")

In [ ]:
from tqdm import tqdm
import json

In [ ]:
all_metrics = []
for i,row in tqdm(df.iterrows()):
  actual = row['Output'].split('[ds]')[1]
  pred = row['model_name']
  # if 'Explanation:' in pred:
  #   pred = pred.split('Explanation:')[1]
  # print(actual)
  # print(pred)
  metric = metrics(actual, pred)
  all_metrics.append(metric)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

317it [01:33,  4.67it/s]Warning: Empty candidate sentence detected; setting raw BERTscores to 0.
397it [01:59,  2.42it/s]Warning: Empty candidate sentence detected; setting raw BERTscores to 0.
559it [02:39,  5.12it/s]Warning: Empty candidate sentence detected; setting raw BERTscores to 0.
647it [03:00,  8.27it/s]Warning: Empty candidate sentence detected; setting raw BERTscores to 0.
761it [03:26,  4.69it/s]Warning: Empty candidate sentence detected; setting raw BERTscores to 0.
928it [04:06,  6.69it/s]Warning: Empty candidate sentence detected; setting raw BERTscores to 0.
933it [04:07,  3.77it/s]


In [ ]:
# all_metrics = []
# for i,row in tqdm(df.iterrows()):
#   actual = row['Output'].split('[ds]')[1]
#   pred = row['llama_val_pe']
#   if '### Explanation:\n' in pred:
#     pred = pred.split('### Explanation:\n')[1]
#   # print(actual)
#   # print(pred)
#   metric = metrics(actual, pred)
#   all_metrics.append(metric)

In [ ]:
import json
with open("/content/drive/MyDrive/PREDEx_gemini_metrics_ft.json", "w") as outfile:
    json.dump(all_metrics, outfile)

In [ ]:
all_metrics

[{'rouge': [{'rouge1': 0.5660731021555764,
    'rouge2': 0.27793427230046946,
    'rougeL': 0.30740393626991563}],
  'bert': 0.6354712843894958,
  'meteor': 0.40788994951980945,
  'bleu': 0.20168909275076702},
 {'rouge': [{'rouge1': 0.21755253399258345,
    'rouge2': 0.19826517967781906,
    'rougeL': 0.1211372064276885}],
  'bert': 0.6526848673820496,
  'meteor': 0.09885170430561417,
  'bleu': 0.0007099832192477313},
 {'rouge': [{'rouge1': 0.528692380056444,
    'rouge2': 0.524033930254477,
    'rougeL': 0.5249294449670743}],
  'bert': 0.8738082051277161,
  'meteor': 0.2831794878523203,
  'bleu': 0.16643719554123398},
 {'rouge': [{'rouge1': 0.23346303501945528,
    'rouge2': 0.2275915822291504,
    'rougeL': 0.23346303501945528}],
  'bert': 0.7090307474136353,
  'meteor': 0.0872527857743877,
  'bleu': 0.0011291499454867753},
 {'rouge': [{'rouge1': 0.6131386861313868,
    'rouge2': 0.6063569682151589,
    'rougeL': 0.6131386861313868}],
  'bert': 0.8047609925270081,
  'meteor': 0.47205

In [ ]:
def avg(l):
  return sum(l)/len(l)

#Rouge

In [ ]:
r1 = []
r2 = []
r3 = []
for m in all_metrics:
  r1.append(m['rouge'][0]['rouge1'])
  r2.append(m['rouge'][0]['rouge2'])
  r3.append(m['rouge'][0]['rougeL'])

In [ ]:
print("Average R1: ", avg(r1))
print("Average R2: ", avg(r2))
print("Average R3: ", avg(r3))

Average R1:  0.3098644794077716
Average R2:  0.2428300203229944
Average R3:  0.2592978679295881


#Blue


In [ ]:
blue = []
for m in all_metrics:
  blue.append(m['bleu'])

In [ ]:
print("Average BLEU: ", avg(blue))

Average BLEU:  0.08261517211624848


#Meteor

In [ ]:
meteor = []
for m in all_metrics:
  meteor.append(m['meteor'])

In [ ]:
print("Average meteor: ", avg(meteor))

Average meteor:  0.18699830594883837


#BERT

In [ ]:
bert = []
for m in all_metrics:
  bert.append(m['bert'])

In [ ]:
print("Average BERT: ", avg(bert))

Average BERT:  0.6328632221472352


#BLANC


In [ ]:
from blanc import BlancHelp, BlancTune

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
bl = BlancHelp(device='cuda', inference_batch_size=128)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def avg(l):
  return sum(l)/len(l)

In [ ]:
def cal_BLANC(actual, pred):
  k = bl.eval_once(actual, pred)
  return k

In [ ]:
all_blanc = []
for i,row in tqdm(df.iterrows()):
  actual = row['Output'].split('[ds]')[1]
  pred = row['gemini_exp'][4:-1]
  metric = cal_BLANC(actual, pred)
  all_blanc.append(metric)

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00,  1.68it/s]
1it [00:00,  1.50it/s]
100%|██████████| 1/1 [00:00<00:00,  3.57it/s]
2it [00:00,  2.17it/s]
100%|██████████| 1/1 [00:00<00:00,  1.43it/s]
3it [00:01,  1.68it/s]
100%|██████████| 1/1 [00:00<00:00,  1.94it/s]
4it [00:02,  1.71it/s]
100%|██████████| 1/1 [00:00<00:00,  6.49it/s]
5it [00:02,  2.28it/s]
100%|██████████| 2/2 [00:00<00:00,  5.07it/s]
6it [00:02,  2.28it/s]
100%|██████████| 1/1 [00:00<00:00,  1.52it/s]
7it [00:03,  1.87it/s]
100%|██████████| 2/2 [00:00<00:00,  2.23it/s]
8it [00:04,  1.48it/s]
100%|██████████| 1/1 [00:00<00:00,  8.16it/s]
9it [00:04,  1.96it/s]
100%|██████████| 1/1 [00:00<00:00,  2.48it/s]
10it [00:05,  2.05it/s]
100%|██████████| 1/1 [00:00<00:00,  1.42it/s]
11it [00:05,  1.73it/s]
100%|██████████| 1/1 [00:00<00:00,  3.53it/s]
12it [00:06,  2.00it/s]
100%|██████████| 1/1 [00:00<00:00,  2.89it/s]
13it [00:06,  2.15it/s]
100%|██████████| 1/1 [00:00<00:00,  7.71it/s]
14it [00:06,  2.69it/s]
100%|█████

In [ ]:
print("Average BLANC: ", avg(all_blanc))

Average BLANC:  0.1714787707684962
